# Hands-on: Data Preparation

## Overview

In this hands-on activity, you will import your dataset (a CSV file) into a notebook to prepare the data. The objective is to guarantee the data quality by pre-processing the raw data before it is utilized in analytics or used as ML model's training dataset.

You will learn about:
1. Process null columns 
2. Process duplicated rows
3. Process outliers
4. Derive new columns 
5. Save cleansed data as new CSV

Sample data: https://ibm.box.com/v/hotel-bookings-sample-dataset

Original data source: 
https://www.sciencedirect.com/science/article/pii/S2352340918315191
https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand

## Setup

In [1]:
# Import library
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)

## Load Data
- Replace this part with your own code. To insert Code Snippet for Data Ingestion, click '</>' icon located in the top-right menu.

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6eOmUMOx-y6TL4Gsb9IOtoI_lEr8d3MP1TAdici3sHFG',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'mlpredictivemodel-donotdelete-pr-hurc6voef6ziuy'
object_key = 'hotel_bookings.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head(10)

hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
0  Resort Hotel            0        342               2015               July   
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   
5  Resort Hotel            0         14               2015               July   
6  Resort Hotel            0          0               2015               July   
7  Resort Hotel            0          9               2015               July   
8  Resort Hotel            1         85               2015               July   
9  Resort Hotel            1         75               2015               July   

   arrival_date_week_number  arrival_date_day_of_month  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   
5                        27                          1   
6                        27                          1   
7                        27                          1   
8                        27                          1   
9                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  adults  children  babies  \
0                        0                     0       2       0.0       0   
1                        0                     0       2       0.0       0   
2                        0                     1       1       0.0       0   
3                        0                     1       1       0.0       0   
4                        0                     2       2       0.0       0   
5                        0                     2       2       0.0       0   
6                        0                     2       2       0.0       0   
7                        0                     2       2       0.0       0   
8                        0                     3       2       0.0       0   
9                        0                     3       2       0.0       0   

  meal country market_segment distribution_channel  is_repeated_guest  \
0   BB     PRT         Direct               Direct                  0   
1   BB     PRT         Direct               Direct                  0   
2   BB     GBR         Direct               Direct                  0   
3   BB     GBR      Corporate            Corporate                  0   
4   BB     GBR      Online TA                TA/TO                  0   
5   BB     GBR      Online TA                TA/TO                  0   
6   BB     PRT         Direct               Direct                  0   
7   FB     PRT         Direct               Direct                  0   
8   BB     PRT      Online TA                TA/TO                  0   
9   HB     PRT  Offline TA/TO                TA/TO                  0   

   previous_cancellations  previous_bookings_not_canceled reserved_room_type  \
0                       0                               0                  C   
1                       0                               0                  C   
2                       0                               0                  A   
3                       0                               0                  A   
4                       0                               0                  A   
5                       0                               0                  A   
6                       0                               0                  C   
7                       0                               0                  C   
8                       0                               0                  A   
9               

In [3]:
# Show number of columns and rows
df.shape

(119390, 32)

In [4]:
# Show all columns name
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [5]:
# Show reservation status unique values
df['reservation_status'].unique()

array(['Check-Out', 'Canceled', 'No-Show'], dtype=object)

## 1. Process Null Columns

In [6]:
# Check null
df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [7]:
# Check columns with null
features = ['children', 'country', 'agent', 'company']

for feat in features:
    perc = len(df[df[feat].isna()])/len(df)*100
    perc = round(perc, 1)
    print(f'Null in {feat}:', perc, '%')
    print(df[feat].describe(), '\n')

Null in children: 0.0 %
count    119386.000000
mean          0.103890
std           0.398561
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          10.000000
Name: children, dtype: float64 

Null in country: 0.4 %
count     118902
unique       177
top          PRT
freq       48590
Name: country, dtype: object 

Null in agent: 13.7 %
count    103050.000000
mean         86.693382
std         110.774548
min           1.000000
25%           9.000000
50%          14.000000
75%         229.000000
max         535.000000
Name: agent, dtype: float64 

Null in company: 94.3 %
count    6797.000000
mean      189.266735
std       131.655015
min         6.000000
25%        62.000000
50%       179.000000
75%       270.000000
max       543.000000
Name: company, dtype: float64 



In [8]:
# For 'children', impute the null with median value
if df['children'].notna().any():
    mode = df['children'].mode()[0]
    df['children'].fillna(value=mode, inplace=True)
    
# For 'country', impute the null with most frequent value
if df['country'].notna().any():
    mode = df['country'].mode()[0]
    df['country'].fillna(value=mode, inplace=True)

# For 'agent', impute the null with most frequent value
if df['agent'].notna().any():
    mode = df['agent'].mode()[0]
    df['agent'].fillna(value=mode, inplace=True)
    
# For 'company', drop the column due to large number of null
df.drop('company', axis=1, inplace=True)

# For 'reservation_status', drop the column due to leakage of our ground truth
df.drop('reservation_status', axis=1, inplace=True)

## 2. Process Duplicated Rows

In [9]:
# Check duplicated data
df[df.duplicated()] 
# -> Since there is no unique reservations ID and the duplicated number of rows is significant, keep the data.

hotel  is_canceled  lead_time  arrival_date_year  \
5       Resort Hotel            0         14               2015   
22      Resort Hotel            0         72               2015   
43      Resort Hotel            0         70               2015   
138     Resort Hotel            1          5               2015   
200     Resort Hotel            0          0               2015   
...              ...          ...        ...                ...   
119349    City Hotel            0        186               2017   
119352    City Hotel            0         63               2017   
119353    City Hotel            0         63               2017   
119354    City Hotel            0         63               2017   
119373    City Hotel            0        175               2017   

       arrival_date_month  arrival_date_week_number  \
5                    July                        27   
22                   July                        27   
43                   July                        27   
138                  July                        28   
200                  July                        28   
...                   ...                       ...   
119349             August                        35   
119352             August                        35   
119353             August                        35   
119354             August                        35   
119373             August                        35   

        arrival_date_day_of_month  stays_in_weekend_nights  \
5                               1                        0   
22                              1                        2   
43                              2                        2   
138                             5                        1   
200                             7                        0   
...                           ...                      ...   
119349                         31                        0   
119352                         31                        0   
119353                         31                        0   
119354                         31                        0   
119373                         31                        1   

        stays_in_week_nights  adults  children  babies meal country  \
5                          2       2       0.0       0   BB     GBR   
22                         4       2       0.0       0   BB     PRT   
43                         3       2       0.0       0   HB     ROU   
138                        0       2       0.0       0   BB     PRT   
200                        1       1       0.0       0   BB     GBR   
...                      ...     ...       ...     ...  ...     ...   
119349                     3       2       0.0       0   BB     DEU   
119352                     3       3       0.0       0   BB     SWE   
119353                     3       3       0.0       0   BB     SWE   
119354                     3       3       0.0       0   BB     SWE   
119373                     3       1       0.0       0   BB     NLD   

       market_segment distribution_channel  is_repeated_guest  \
5           Online TA                TA/TO                  0   
22             Direct               Direct                  0   
43             Direct               Direct                  0   
138         Online TA                TA/TO                  0   
200         Online TA                TA/TO                  0   
...               ...                  ...                ...   
119349      Online TA                TA/TO                  0   
119352      Online TA                TA/TO                  0   
119353      Online TA                TA/TO                  0   
119354      Online TA                TA/TO                  0   
119373  Offline TA/TO                TA/TO                  0   

        previous_cancellations  previous_bookings_not_canceled  \
5                            0                               0   
22                           0        

## 3. Process Outliers

In [10]:
# Check outliers in original data
df_describe = pd.DataFrame(df.describe(include='all'))
df_describe

hotel    is_canceled      lead_time  arrival_date_year  \
count       119390  119390.000000  119390.000000      119390.000000   
unique           2            NaN            NaN                NaN   
top     City Hotel            NaN            NaN                NaN   
freq         79330            NaN            NaN                NaN   
mean           NaN       0.370416     104.011416        2016.156554   
std            NaN       0.482918     106.863097           0.707476   
min            NaN       0.000000       0.000000        2015.000000   
25%            NaN       0.000000      18.000000        2016.000000   
50%            NaN       0.000000      69.000000        2016.000000   
75%            NaN       1.000000     160.000000        2017.000000   
max            NaN       1.000000     737.000000        2017.000000   

       arrival_date_month  arrival_date_week_number  \
count              119390             119390.000000   
unique                 12                       NaN   
top                August                       NaN   
freq                13877                       NaN   
mean                  NaN                 27.165173   
std                   NaN                 13.605138   
min                   NaN                  1.000000   
25%                   NaN                 16.000000   
50%                   NaN                 28.000000   
75%                   NaN                 38.000000   
max                   NaN                 53.000000   

        arrival_date_day_of_month  stays_in_weekend_nights  \
count               119390.000000            119390.000000   
unique                        NaN                      NaN   
top                           NaN                      NaN   
freq                          NaN                      NaN   
mean                    15.798241                 0.927599   
std                      8.780829                 0.998613   
min                      1.000000                 0.000000   
25%                      8.000000                 0.000000   
50%                     16.000000                 1.000000   
75%                     23.000000                 2.000000   
max                     31.000000                19.000000   

        stays_in_week_nights         adults       children         babies  \
count          119390.000000  119390.000000  119390.000000  119390.000000   
unique                   NaN            NaN            NaN            NaN   
top                      NaN            NaN            NaN            NaN   
freq                     NaN            NaN            NaN            NaN   
mean                2.500302       1.856403       0.103886       0.007949   
std                 1.908286       0.579261       0.398555       0.097436   
min                 0.000000       0.000000       0.000000       0.000000   
25%                 1.000000       2.000000       0.000000       0.000000   
50%                 2.000000       2.000000       0.000000       0.000000   
75%                 3.000000       2.000000       0.000000       0.000000   
max                50.000000      55.000000      10.000000      10.000000   

          meal country market_segment distribution_channel  is_repeated_guest  \
count   119390  119390         119390               119390      119390.000000   
unique       5     177              8                    5                NaN   
top         BB     PRT      Online TA                TA/TO                NaN   
freq     92310   49078          56477                97870                NaN   
mean       NaN     NaN            NaN                  NaN           0.031912   
std        NaN     NaN            NaN                  NaN           0.175767   
min        NaN     NaN            NaN                  NaN           0.000000   
25%        NaN     NaN            NaN                  NaN           0.000000   
50%        NaN     NaN            NaN                  NaN           0.000000   
75%        NaN     

In [11]:
# Process outliers in original data

# Impute outlier value with 0
df.loc[df['adults']>4, 'adults'] = 0
df.loc[df['children']>4, 'children'] = 0
df.loc[df['babies']>4, 'babies'] = 0

# 'Meal' contains values "Undefined", which is equal to SC
df['meal'].replace('Undefined', 'SC', inplace=True)

## 4. Derive New Columns

In [12]:
# Derive new columns

# Create 'total_stay_nights'
df['total_stay_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

# Create 'kids'& 'num_pax'
df['kids'] = df['children'] + df['babies'] 
df['num_pax'] = df['adults'] + df['kids'] 

In [13]:
# Check outliers in derived data
df_der = pd.DataFrame(df[['total_stay_nights', 'num_pax']].describe())
df_der

total_stay_nights        num_pax
count      119390.000000  119390.000000
mean            3.427900       1.964687
std             2.557439       0.651295
min             0.000000       0.000000
25%             2.000000       2.000000
50%             3.000000       2.000000
75%             4.000000       2.000000
max            69.000000       5.000000

In [14]:
print(df[df['num_pax']==0].shape)
# Drop the rows if 'num_pax' == 0
df = df[df['num_pax']!=0]
df.shape
# -> Remove the rows since there is no data about pax and the row number is not significant

(196, 33)


(119194, 33)

## 5. Save Cleansed Data as New CSV

In [15]:
#The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform API
from project_lib import Project

project = Project(None, '<my_project_id>', '<my_project_token>')
pc = project.project_context

# Show Project, Bucket and Assets
print('Project Name: {0}'.format(project.get_name()))
print('Project Description: {0}'.format(project.get_description()))
print('Project Bucket Name: {0}'.format(project.get_project_bucket_name()))
print('Project Assets (Connections): {0}'.format(project.get_assets(asset_type='connection')))

# Save dataframe as csv file in your bucket 
project.save_data(data=df.to_csv(index=False), file_name='hotel_bookings_v1.csv', overwrite=True)

Project Name: ML Predictive Model
Project Description: 
Project Bucket Name: mlpredictivemodel-donotdelete-pr-hurc6voef6ziuy
Project Assets (Connections): []


{'file_name': 'hotel_bookings_v1.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'mlpredictivemodel-donotdelete-pr-hurc6voef6ziuy',
 'asset_id': '90706867-09bd-4700-9a34-8af199e792a7'}

## Summary 

In this hands-on activity, you have covered the following:

1. Checked the quality of the data.
2. Conducted data wrangling to ensure datasets were of acceptable quality for use in exploratory data analysis (EDA) and Machine Learning (ML) model development.
3. Saved the cleansed dataset into a new CSV file.